In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds1 = open_mdsdataset(data_dir, geometry='cartesian', endian='<',prefix=['statevars'],iters=range(120*1860,170*1860,1860))

/scratch/jxchang/HighRes1/results/TideU008N0SimH200ho140Ah0200Cdqdt003_advSch2/input/


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.animation as animation
import math
%matplotlib inline

In [3]:
t=0
u0=0.08
om=2*np.pi/12.4/3600 


vmax = 3
vmin = -3

numcolt=21
numcolv=30

ttlen=len(ds1.time)
print('the length of time:' + str(ttlen) )

the length of time:53


In [4]:
xg=ds1.coords["XG"]
xc=ds1.coords["XC"]
yg=ds1.coords["YG"]
yc=ds1.coords["YC"]
z=ds1.coords["Z"]
time=ds1.coords['time'].values/np.timedelta64(1, 's')/3600
yc0 = (ds1.YC/1000.-ds1.YC.mean()/1000.)
xc0 = (ds1.XC/1000.-ds1.XC.mean()/1000.)

In [36]:
fig = plt.figure(figsize=(11,8))
grid = plt.GridSpec(2, 2, height_ratios=[0.6,2.5], width_ratios=[1,0.1],wspace=0.02,hspace=0.38)

<Figure size 792x576 with 0 Axes>

In [37]:
SMALL_SIZE = 17
MEDIUM_SIZE = 19
BIGGER_SIZE = 21

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [38]:
#plot bathymetry
ax1=fig.add_subplot(grid[0,:-1])
ax1.plot(time,np.zeros(ttlen),color='gray',linestyle='--')
ax1.plot(time,ds1.UVEL.isel(Z=0,YC=0,XG=0),label='U(t,0,0,0)')
ax1.plot(time,u0*np.cos(om*ds1.time.values/np.timedelta64(1, 's')),label='Uinput')
ax1.set_xlim(min(time), max(time))
ax1.set_ylim(-u0-0.05, u0+0.05)
point, = ax1.plot([time[0]],[ds1.UVEL.isel(time=t,YC=40,XG=0,Z=0).data], 'go', color='red',ms=10)
ax1.set_xlabel('time [hr]')
ax1.set_ylabel('U [m/s]')
ax1.set_title('Uinput vs U[t,0,0,0] ', loc='left')



Text(0.0, 1.0, 'Uinput vs U[t,0,0,0] ')

In [39]:
dmax=10**(-1)
dmin=10**(-6)
xmin = -4
xmax = 4

# plot epsh
ax2=fig.add_subplot(grid[1,0])

pcm=ax2.pcolormesh(xc0,yc0, ds1.KLeps.isel(time=t,Zl=6), vmax=dmax, vmin=dmin
                ,norm=colors.LogNorm(vmin=dmin, vmax=dmax),cmap='RdYlBu_r',rasterized=True)
#cset1 = ax2.contourf(ds1.XC0,  hDispbc.isel(time=t,Z=6))

cax = fig.add_subplot(grid[1,-1])
cax.set_visible(False)
fig.colorbar(pcm, ax=cax,extend='both')
#fig.colorbar(cset1, ax=axs,ticks=np.arange(tmin, tmax+0.1, 5))
ax2.set_xlabel('X [km]')
ax2.set_ylabel('Y [km]')
ax2.set_title(r'$\epsilon_v$', loc='left')
ax2.set(xlim=(xmin, xmax))
#ax2.plot(ds1['XC0'],-ds1.Depth.isel(YC=60),lw=1.5)

/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  # Remove the CWD from sys.path while we load stuff.
/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  # Remove the CWD from sys.path while we load stuff.


[(-10.0, 10.0)]

In [40]:
fig
print(ax2.collections)
print(np.size(ax2.collections))

1


In [41]:
tt: int=0
def updateData(tt):

    print(tt)
    updatetime = time[tt]
    pcm.set_array(ds1.KLeps[tt,6,:-1, :-1].values.flatten())
    point.set_data(updatetime,ds1.UVEL.isel(time=tt,YC=60,XG=0,Z=0))
    ax1.set_title('t= %2.3fhr' % updatetime)
    fig1 = plt.gcf()
    fig.savefig('./figsMag2/summary_epsv_z6_xy_t%s.png' %(tt+120))



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= 40 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_epsv_z6_xy.gif', writer='imagemagick', fps=2)

<Figure size 432x288 with 0 Axes>

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


<Figure size 432x288 with 0 Axes>

In [42]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_epsv_z6_xy.gif'))